# Basic simulation of electrodes in ESPResSo part II:
# Electrolyte capacitor and Poisson--Boltzmann theory

In this tutorial we are going to investigate [...] using **ESPResSo**. 

To work with this tutorial, you should be familiar with the following topics:

1. Setting up and running simulations in ESPResSo - creating particles,
   incorporating interactions.
   If you are unfamiliar with this, you can go through the respective tutorial
   in the `lennard_jones` folder.
2. Basic knowledge of classical electrostatics:
   Dipoles, surface and image charges
3. Reduced units, as described in the ESPResSo user guide
   https://espressomd.github.io/doc/introduction.html#on-units

## Introduction

Understanding the electric double layer (EDL) is crucial for the study
of a variety of systems, including colloidal suspensions, charged biological
macromolecules and membranes. Poisson-Boltzmann (PB) theory based on a mean-field formalism properly describes the behavior of Coulomb fluids composed of monovalent ions at low concentrations in the
vicinity of weakly charged interfaces. However, for strongly charged systems, where correlation and finite size effects begin to dominate the system dynamics, the PB theory falls inadequate. Our goal in this module is to demonstrate how coarse grained implicit solvent simualtions can corroborate some of these approximate theories.

Here, we look at the ion density profile between two dielectric walls with constant potential applied to them. 
In this case the inclusion of dielectric inhomogeneities demands a detailed calculation of the image
effects that involve the full solution of the Poisson equation on the fly. This is dealt in a computational cost effective way using ELCIC method, to treat the image charge effect in the presence of 2D dielectric bounding
interfaces.  

[...]

## Theoretical Background 

[...]

##  System setup 

First we import all ESPResSo features and external modules

In [ ]:
import espressomd
import numpy as np
import espressomd.electrostatics
import espressomd.electrostatic_extensions
from espressomd.interactions import *
import espressomd.visualization
import espressomd.observables
import espressomd.accumulators
from espressomd import shapes
import matplotlib.pyplot as plt
from scipy.special import *
from scipy import constants as const
from tqdm import tqdm


We define system dimensions and some physical parameters related to length, time
and energy scales of our system.
All physical parameters are defined in reduced units of length ($\sigma=1$;
Particle size), mass ($m=1$; Particle mass), time ($t=0.01 \tau$) and
elementary charge ($e=1$).

Another important length scale is the Bjerrum Length, which is the length at
which the electrostatic energy between two elementary charges is comparable to
the thermal energy $k_\mathrm{B}T$.
It is defined as $l_\mathrm{B}=\frac{1}{4\pi\epsilon_0\epsilon_rk_BT}$. 
In our case if we choose the ion size ($\sigma$) in simulations equivalent to a
typical experimental value for mono-atomar salt, 0.3 nm in real units, then the
Bjerrum length of water at room temperature, $\ell_\mathrm{B}=0.71 \,\mathrm{nm}$ is
$\ell_\mathrm{B}\sim 2$ in simulations units.

In [ ]:
# Keep lb=2 (water) and LJ units, convert to physical units using ION_SIZE

BJERRUM_LENGTH = 0.71        # Electrostatic prefactor passed to P3M ; prefactor=lB KBT/e2                

#Lennard-Jones  Parameters
LJ_SIGMA = 0.3              # Particle size nanometers
LJ_EPSILON = 1.0

CONCENTRATION = 1e-3 # desired concentration 1 mmol/l
DISTANCE = 200 # 20 Debye lengths
N_IONPAIRS = 2000

# Elementary charge 
q = np.array([1.0])  
types = {"Cation": 0, "Anion": 1  ,"Electrodes": 2}
charges = {"Cation": q[0], "Anion": -q[0]  }

In [ ]:
def get_box_dimension(concentration, distance, n_ionpairs=N_IONPAIRS):
    """ For a given number of particles, determine the lateral area of the box
    to match the desired concentration """

    # concentration is in mol/l, convert to 1/sigma**3
    rho = concentration * (const.Avogadro / const.liter) * const.nano**3

    box_volume = n_ionpairs / rho
    area = box_volume / distance
    l_xy = np.sqrt(area)

    return rho,l_xy
    # determine the box size to achieve the desired density
    #return (rho * distance**2)**(1.0/3.0)

rho, l_xy = get_box_dimension(CONCENTRATION,DISTANCE,N_IONPAIRS)
DEBYE_LENGTH = (4 * np.pi * BJERRUM_LENGTH * rho*2)**(-1./2) # desired Debye length in nm

In [ ]:
ELC_GAP = 0.3*DISTANCE
system = espressomd.System(box_l=[l_xy, l_xy, DISTANCE+ELC_GAP])
system.time_step = 0.01

We now set up an electrolyte solution made of monovalent cations (Np) and anions
(Np) between two metallic electrodes at constant potential. 
Firstly, we add two wall constraints at $z=0$ and $z=L_z$ to stop particles from
crossing the boundaries.
Refer to  [espressomd.constraints.ShapeBasedConstraint]
(https://espressomd.github.io/doc/espressomd.html#espressomd.constraints.ShapeBasedConstraint)
in the documentation to set up constraints. Then we place all the ion pairs at
random positions between the electrodes.  

In [ ]:
# ADDING ELECTRODES
wall_offset=0

# (BOTTOM WALL) normal pointing in the +z direction, at z=wall_offset 
floor = espressomd.shapes.Wall(normal=[0, 0, 1], dist=wall_offset)
c1 = system.constraints.add(
    particle_type=types["Electrodes"], penetrable=True, only_positive=False, shape=floor)

# (TOP WALL) normal pointing in the -z direction, at z=box_l_z-wall_offset
ceil = espressomd.shapes.Wall(normal=[0, 0, -1], dist=-(DISTANCE - wall_offset))   
c2 = system.constraints.add(
    particle_type=types["Electrodes"], penetrable=True, only_positive=False, shape=ceil)


# ADDING IONPAIRS 
offset=10 # To stop the particle to cross the wall at integration 
Init_part_btw_z1=0+offset 
Init_part_btw_z2=DISTANCE-offset
ion_pos=np.empty((3),dtype=float)

for i in range (N_IONPAIRS):
    ion_pos[0] = np.random.random(1) * system.box_l[0]
    ion_pos[1] = np.random.random(1) * system.box_l[1]
    ion_pos[2] = np.random.random(1) * (Init_part_btw_z2-Init_part_btw_z1) + Init_part_btw_z1
    system.part.add(pos=ion_pos, type=types["Cation"]  , q=charges["Cation"])
    
for i in range (N_IONPAIRS):
    ion_pos[0] = np.random.random(1) * system.box_l[0]
    ion_pos[1] = np.random.random(1) * system.box_l[1]
    ion_pos[2] = np.random.random(1) * (Init_part_btw_z2-Init_part_btw_z1) + Init_part_btw_z1
    system.part.add(pos=ion_pos, type=types["Anion"]  , q=charges["Anion"])

Now, we add interactions:
For excluded volume interactions, we add WCA kind of potential. 
For the (2D+h) electrostatic with dielectrics we choose the ELCIC with P3M.

Refer to the documentation to set up the
[WCA interaction](https://espressomd.github.io/doc/espressomd.html#espressomd.interactions.WCAInteraction) 
under [Non-bonded](https://espressomd.github.io/doc/inter_non-bonded.html)
section.

Refer the documentation to set up
[ELCIC with P3M](https://espressomd.github.io/doc/electrostatics.html#electrostatic-layer-correction-elc)
under the [electrostatics](https://espressomd.github.io/doc/electrostatics.html)
section. 

In [ ]:
#***************************************************************
#                   Adding WCA interactions 
#***************************************************************

for  key, val in types.items():
    for key1, val1 in types.items():
        system.non_bonded_inter[val, val1].wca.set_params(epsilon=LJ_EPSILON, sigma=LJ_SIGMA)

#***************************************************************
#                 Adding ELC-P3M electrostatic solver   
#***************************************************************

delta_mid_top = -1.0   #(Fully metallic case both -1)                 
delta_mid_bot = -1.0
potential_diff=4.0
accuracy = 1e-3
check_accuracy = 1e-3
p3m = espressomd.electrostatics.P3M(prefactor=BJERRUM_LENGTH, accuracy=accuracy, mesh=[58, 58, 70], cao=4)
elc = espressomd.electrostatics.ELC(actor=p3m,
                                    gap_size=ELC_GAP,
                                    const_pot=True,
                                    pot_diff=potential_diff,
                                    maxPWerror=check_accuracy,
                                    delta_mid_bot=delta_mid_bot,
                                    delta_mid_top=delta_mid_top)
system.electrostatics.solver = elc

Before we can start the simulation, we need to remove the overlap between particles to avoid large forces which would crash the simulation. For this, we use the steepest descent integrator with a relative convergence criterion for forces and energies.



In [ ]:
#  Relax the overlaps with steepest descent

system.integrator.set_steepest_descent(f_max=10, gamma=50.0,
                                       max_displacement=0.02)
system.integrator.run(1000)
system.integrator.set_vv() # Switch bach to velocity Verlet 


# Add thermostat 
thermostat_seed = np.random.randint(np.random.randint(1000000))
system.thermostat.set_langevin(kT=1.0, gamma=0.1, seed=thermostat_seed)

Convergence after $t=750$ time units, possible to run up to $t=1000$ or $t=2000$ here?
This is a total of 100.000 or 200.000 time steps, respectively.

In [ ]:
# Integration parameters
STEPS_PER_SAMPLE = 250
N_SAMPLES = 10
N_PART = 2* N_IONPAIRS

times = []
e_total = []

In [ ]:
for i in tqdm(range(N_SAMPLES)):
    times[i] = system.time
    energy = system.analysis.energy()
    e_total[i] = energy['total']
    system.integrator.run(STEPS_PER_SAMPLE)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(times, e_total)
plt.xlabel('t')
plt.ylabel('E')
plt.show()

In [ ]:
#*************************************************************
#  PART 2:       SETTING ACCUMULATORS FOR DENSITY PROFILE 
#**************************************************************
# PRODUCTION RUNS
Ion_id=[]
Cations = system.part.select(type=types["Cation"])
Cations_id=[]
for i in Cations:
    Cations_id.append(i.id)
    Ion_id.append(i.id)
    
Anions = system.part.select(type=types["Anion"])
Anions_id=[]
for i in Anions:
    Anions_id.append(i.id)
    Ion_id.append(i.id)
    

  
# Accumulator 1 : observable::Density_Profile
density_profile_cation = espressomd.observables.DensityProfile(ids=Cations_id,
                                                       n_x_bins=1,
                                                       n_y_bins=1,
                                                       n_z_bins=2*int(system.box_l[2] - ELC_GAP),
                                                       min_x=0,
                                                       min_y=0,
                                                       min_z=0,
                                                       max_x=system.box_l[0],
                                                       max_y=system.box_l[1],
                                                       max_z=system.box_l[2] - ELC_GAP)

density_accumulator_cation = espressomd.accumulators.MeanVarianceCalculator(obs=density_profile_cation, delta_N=20)


density_profile_anion = espressomd.observables.DensityProfile(ids=Anions_id,
                                                       n_x_bins=1,
                                                       n_y_bins=1,
                                                       n_z_bins=2*int(system.box_l[2] - ELC_GAP),
                                                       min_x=0,
                                                       min_y=0,
                                                       min_z=0,
                                                       max_x=system.box_l[0],
                                                       max_y=system.box_l[1],
                                                       max_z=system.box_l[2] - ELC_GAP)

density_accumulator_anion = espressomd.accumulators.MeanVarianceCalculator(obs=density_profile_anion, delta_N=20)


system.auto_update_accumulators.clear()
system.auto_update_accumulators.add(density_accumulator_cation)
system.auto_update_accumulators.add(density_accumulator_anion)
    
times=[]
e_total=[]
for tm in tqdm(range(N_SAMPLES)):
    system.integrator.run(STEPS_PER_SAMPLE)
    times.append( system.time)
    energy = system.analysis.energy()
    e_total.append( energy['total'])   

In [ ]:
zs = density_profile_anion.bin_centers()[0, 0, :, 2]
cation_profile_mean = density_accumulator_cation.mean()[0, 0, :]
anion_profile_mean = density_accumulator_anion.mean()[0, 0, :]

In [ ]:
plt.plot(zs, cation_profile_mean)
plt.plot(zs, anion_profile_mean)
plt.plot(zs, cation_profile_mean + anion_profile_mean, color='k')
plt.yscale('log')

## References

<a id='[1]'></a>[1] 
